In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/drive/MyDrive/JU PROJ/Reddit/EXEC_RedditTesla.csv')
df

,upvotes,Date,Post,sub
0,1,2022-05-20,I have been investing in Tesla for awhile. My ...,tsla
1,1,2022-05-19,"Woes to TSLA stock politically, starting from ...",tsla
2,1,2022-05-19,The sell off stocks of companies.like Apple &a...,tsla
3,1,2022-05-11,Woes to TSLA. Musk is fighting against some bi...,tsla
4,1,2022-05-11,Im looking to buy my first tsla shares but hav...,tsla
...,...,...,...,...
1234,1,2019-04-18,My TSLA peeps when do you guys expect it to go...,Tesla_Stock
1235,1,2022-05-11,Hey where is everybody? Is it happening? \n\nI...,tslaq
1236,1,2022-01-13,That's hilarious you fucking losers. LMAO! I h...,tslaq
1237,1,2021-05-10,Muckster the huckster called it a hustle on li...,tslaq


In [6]:
df = df.drop(['upvotes', 'sub'], axis = 1)

In [7]:
df['Date'] = pd.to_datetime(df['Date']).dt.date
df

,Date,Post
0,2022-05-20,I have been investing in Tesla for awhile. My ...
1,2022-05-19,"Woes to TSLA stock politically, starting from ..."
2,2022-05-19,The sell off stocks of companies.like Apple &a...
3,2022-05-11,Woes to TSLA. Musk is fighting against some bi...
4,2022-05-11,Im looking to buy my first tsla shares but hav...
...,...,...
1234,2019-04-18,My TSLA peeps when do you guys expect it to go...
1235,2022-05-11,Hey where is everybody? Is it happening? \n\nI...
1236,2022-01-13,That's hilarious you fucking losers. LMAO! I h...
1237,2021-05-10,Muckster the huckster called it a hustle on li...


In [8]:
df.columns = ['Date', 'Tweet']
df

,Date,Tweet
0,2022-05-20,I have been investing in Tesla for awhile. My ...
1,2022-05-19,"Woes to TSLA stock politically, starting from ..."
2,2022-05-19,The sell off stocks of companies.like Apple &a...
3,2022-05-11,Woes to TSLA. Musk is fighting against some bi...
4,2022-05-11,Im looking to buy my first tsla shares but hav...
...,...,...
1234,2019-04-18,My TSLA peeps when do you guys expect it to go...
1235,2022-05-11,Hey where is everybody? Is it happening? \n\nI...
1236,2022-01-13,That's hilarious you fucking losers. LMAO! I h...
1237,2021-05-10,Muckster the huckster called it a hustle on li...


In [9]:
def tweets_per_day(df):
    df['Date'] = pd.to_datetime(df['Date'])
#   return df[['Tweets']].groupby(df['Date'].dt.date).count()
    tweet = df['Tweet'].groupby(df['Date'].dt.date).count() 
    d1 = pd.DataFrame(tweet)
    return d1
  # if you want output to be `Series`

df1 = tweets_per_day(df)
df1 = df1.reset_index(level=0)
df1

,Date,Tweet
0,2018-07-27,1
1,2018-08-17,1
2,2018-08-19,5
3,2018-08-20,2
4,2018-08-23,1
...,...,...
667,2022-05-14,1
668,2022-05-19,2
669,2022-05-20,2
670,2022-05-22,1


In [10]:
#df = df.groupby(['Date'])['Tweet'].apply(lambda x: '. '.join(x.astype(str))).reset_index()
df

,Date,Tweet
0,2022-05-20,I have been investing in Tesla for awhile. My ...
1,2022-05-19,"Woes to TSLA stock politically, starting from ..."
2,2022-05-19,The sell off stocks of companies.like Apple &a...
3,2022-05-11,Woes to TSLA. Musk is fighting against some bi...
4,2022-05-11,Im looking to buy my first tsla shares but hav...
...,...,...
1234,2019-04-18,My TSLA peeps when do you guys expect it to go...
1235,2022-05-11,Hey where is everybody? Is it happening? \n\nI...
1236,2022-01-13,That's hilarious you fucking losers. LMAO! I h...
1237,2021-05-10,Muckster the huckster called it a hustle on li...


In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [12]:
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
import math

stop_words = stopwords.words('english')
stop_words.remove("not")
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

def preprocess(text, stem=False):
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)

df.Tweets = df.Tweet.apply(lambda x: preprocess(x))
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


,Date,Tweet
0,2022-05-20,I have been investing in Tesla for awhile. My ...
1,2022-05-19,"Woes to TSLA stock politically, starting from ..."
2,2022-05-19,The sell off stocks of companies.like Apple &a...
3,2022-05-11,Woes to TSLA. Musk is fighting against some bi...
4,2022-05-11,Im looking to buy my first tsla shares but hav...
...,...,...
1234,2019-04-18,My TSLA peeps when do you guys expect it to go...
1235,2022-05-11,Hey where is everybody? Is it happening? \n\nI...
1236,2022-01-13,That's hilarious you fucking losers. LMAO! I h...
1237,2021-05-10,Muckster the huckster called it a hustle on li...


In [13]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [14]:
neg = []
neu = []
pos = []
i = 0
def normalize(score, alpha=15):
    """
    Normalize the score to be between -1 and 1 using an alpha that
    approximates the max expected value
    """
    norm_score = score/math.sqrt((score*score) + alpha)
    return norm_score

for tweet in df.Tweet:
  i = i+1
  encoded_tweet = tokenizer(tweet,padding=True, truncation=True,max_length=250, add_special_tokens = True, return_tensors='pt')
  output = model(**encoded_tweet)
  scores = output[0][0].detach().numpy()
  normScores = softmax(scores)  
  neg.append(normScores[0])
  neu.append(normScores[1])
  pos.append(normScores[2])
  print(i,"--> ",normScores)

1 -->  [0.03003492 0.35369897 0.61626613]
2 -->  [0.03609845 0.21974905 0.74415237]
3 -->  [0.81567514 0.02419174 0.16013299]
4 -->  [0.1645294  0.45887    0.37660062]
5 -->  [0.07062847 0.2714281  0.65794355]
6 -->  [0.08546304 0.01483318 0.89970374]
7 -->  [0.23408812 0.02356591 0.7423459 ]
8 -->  [0.04657635 0.04088338 0.9125404 ]
9 -->  [0.0194683  0.06512311 0.9154085 ]
10 -->  [0.04637482 0.03331199 0.92031324]
11 -->  [0.06268669 0.02388318 0.9134302 ]
12 -->  [0.12944436 0.04362182 0.82693386]
13 -->  [0.01902589 0.10516563 0.87580866]
14 -->  [0.01859744 0.71346945 0.26793313]
15 -->  [0.00995922 0.9693178  0.02072302]
16 -->  [0.07229238 0.0642685  0.86343914]
17 -->  [0.02819197 0.01899362 0.95281446]
18 -->  [0.42490122 0.034032   0.54106677]
19 -->  [0.766927   0.01164385 0.22142906]
20 -->  [0.18336876 0.44263452 0.37399676]
21 -->  [0.01851944 0.7996903  0.18179017]
22 -->  [0.0264072  0.02555989 0.94803303]
23 -->  [0.02895524 0.08744634 0.88359845]
24 -->  [0.04264919 

In [15]:
df['negative'] = neg
df['neutral'] = neu
df['positive'] = pos
df

,Date,Tweet,negative,neutral,positive
0,2022-05-20,I have been investing in Tesla for awhile. My ...,0.030035,0.353699,0.616266
1,2022-05-19,"Woes to TSLA stock politically, starting from ...",0.036098,0.219749,0.744152
2,2022-05-19,The sell off stocks of companies.like Apple &a...,0.815675,0.024192,0.160133
3,2022-05-11,Woes to TSLA. Musk is fighting against some bi...,0.164529,0.458870,0.376601
4,2022-05-11,Im looking to buy my first tsla shares but hav...,0.070628,0.271428,0.657944
...,...,...,...,...,...
1234,2019-04-18,My TSLA peeps when do you guys expect it to go...,0.215119,0.018619,0.766263
1235,2022-05-11,Hey where is everybody? Is it happening? \n\nI...,0.008807,0.946714,0.044480
1236,2022-01-13,That's hilarious you fucking losers. LMAO! I h...,0.039357,0.060266,0.900377
1237,2021-05-10,Muckster the huckster called it a hustle on li...,0.091650,0.130431,0.777919


In [16]:
dfx = pd.DataFrame()
po = []
nu = []
ne = []
for i in range (0,672):

  d = df['Date'][i]

  x = df[df['Date'] == d]
  po.append(x['positive'].mean())
  nu.append(x['neutral'].mean())
  ne.append(x['negative'].mean())
dfx['Date'] = df1['Date']
dfx['positive'] = po
dfx['negative'] = ne
dfx['neutral'] = nu
dfx

,Date,positive,negative,neutral
0,2018-07-27,0.712959,0.029995,0.257046
1,2018-08-17,0.452143,0.425887,0.121970
2,2018-08-19,0.452143,0.425887,0.121970
3,2018-08-20,0.359675,0.081321,0.559004
4,2018-08-23,0.359675,0.081321,0.559004
...,...,...,...,...
667,2022-05-14,0.918415,0.052354,0.029232
668,2022-05-19,0.806404,0.166480,0.027116
669,2022-05-20,0.806404,0.166480,0.027116
670,2022-05-22,0.871622,0.051960,0.076418


In [17]:
# Set date as Index
dfx = dfx.set_index('Date')
dfx = dfx.sort_index()
dfx

,positive,negative,neutral
Date,,,
2018-07-27,0.712959,0.029995,0.257046
2018-08-17,0.452143,0.425887,0.121970
2018-08-19,0.452143,0.425887,0.121970
2018-08-20,0.359675,0.081321,0.559004
2018-08-23,0.359675,0.081321,0.559004
...,...,...,...
2022-05-14,0.918415,0.052354,0.029232
2022-05-19,0.806404,0.166480,0.027116
2022-05-20,0.806404,0.166480,0.027116


### STOCK DATA

In [18]:
from pandas_datareader import data as pdr
import yfinance as yf

yf.pdr_override()

stock_price = pdr.get_data_yahoo("TSLA", start = '2017-01-01', end = '2022-05-07')
print(len(stock_price))
stock_price

[*********************100%***********************]  1 of 1 completed
1346


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,42.972000,44.066002,42.192001,43.397999,43.397999,29616500
2017-01-04,42.950001,45.599998,42.862000,45.397999,45.397999,56067500
2017-01-05,45.284000,45.495998,44.389999,45.349998,45.349998,29558500
2017-01-06,45.386002,46.062000,45.090000,45.801998,45.801998,27639500
2017-01-09,45.793999,46.383999,45.599998,46.256001,46.256001,19897500
...,...,...,...,...,...,...
2022-05-02,860.770020,906.359985,848.030029,902.940002,902.940002,25260500
2022-05-03,903.179993,924.080017,888.590027,909.250000,909.250000,21236500
2022-05-04,903.940002,955.500000,885.280029,952.619995,952.619995,27214600


In [19]:
stock_price = stock_price.drop_duplicates()

# dropping the zero values from volume
stock_price = stock_price[stock_price['Volume'] != 0]

# filtering the important columns required
stock_price = stock_price.filter(['Date', 'Close', 'Open', 'High', 'Low', 'Volume'])

# sorting the data according to the index i.e 'Date'
stock_price = stock_price.sort_index(ascending=True, axis=0)
stock_price

,Close,Open,High,Low,Volume
Date,,,,,
2017-01-03,43.397999,42.972000,44.066002,42.192001,29616500
2017-01-04,45.397999,42.950001,45.599998,42.862000,56067500
2017-01-05,45.349998,45.284000,45.495998,44.389999,29558500
2017-01-06,45.801998,45.386002,46.062000,45.090000,27639500
2017-01-09,46.256001,45.793999,46.383999,45.599998,19897500
...,...,...,...,...,...
2022-05-02,902.940002,860.770020,906.359985,848.030029,25260500
2022-05-03,909.250000,903.179993,924.080017,888.590027,21236500
2022-05-04,952.619995,903.940002,955.500000,885.280029,27214600


### COMBINE DATA

In [20]:
stock_data = pd.merge(stock_price,dfx, how='left', left_index=True, right_index=True)

stock_data

,Close,Open,High,Low,Volume,positive,negative,neutral
Date,,,,,,,,
2017-01-03,43.397999,42.972000,44.066002,42.192001,29616500,NaN,NaN,NaN
2017-01-04,45.397999,42.950001,45.599998,42.862000,56067500,NaN,NaN,NaN
2017-01-05,45.349998,45.284000,45.495998,44.389999,29558500,NaN,NaN,NaN
2017-01-06,45.801998,45.386002,46.062000,45.090000,27639500,NaN,NaN,NaN
2017-01-09,46.256001,45.793999,46.383999,45.599998,19897500,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-05-02,902.940002,860.770020,906.359985,848.030029,25260500,0.888990,0.094719,0.016290
2022-05-03,909.250000,903.179993,924.080017,888.590027,21236500,0.827792,0.160966,0.011242
2022-05-04,952.619995,903.940002,955.500000,885.280029,27214600,NaN,NaN,NaN


In [21]:
stock_data.sort_index(axis = 1)
stock_data

,Close,Open,High,Low,Volume,positive,negative,neutral
Date,,,,,,,,
2017-01-03,43.397999,42.972000,44.066002,42.192001,29616500,NaN,NaN,NaN
2017-01-04,45.397999,42.950001,45.599998,42.862000,56067500,NaN,NaN,NaN
2017-01-05,45.349998,45.284000,45.495998,44.389999,29558500,NaN,NaN,NaN
2017-01-06,45.801998,45.386002,46.062000,45.090000,27639500,NaN,NaN,NaN
2017-01-09,46.256001,45.793999,46.383999,45.599998,19897500,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-05-02,902.940002,860.770020,906.359985,848.030029,25260500,0.888990,0.094719,0.016290
2022-05-03,909.250000,903.179993,924.080017,888.590027,21236500,0.827792,0.160966,0.011242
2022-05-04,952.619995,903.940002,955.500000,885.280029,27214600,NaN,NaN,NaN


In [22]:
stock_data.isna().sum()

Close         0
Open          0
High          0
Low           0
Volume        0
positive    854
negative    854
neutral     854
dtype: int64

In [23]:
#stock_data.drop(['Tweet'], inplace=True, axis=1)

# rearranging the columns of the whole stock_data
stock_data = stock_data[['Close', 'negative', 'neutral', 'positive', 'Open', 'High', 'Low', 'Volume']]

# displaying the final stock_data
stock_data

,Close,negative,neutral,positive,Open,High,Low,Volume
Date,,,,,,,,
2017-01-03,43.397999,NaN,NaN,NaN,42.972000,44.066002,42.192001,29616500
2017-01-04,45.397999,NaN,NaN,NaN,42.950001,45.599998,42.862000,56067500
2017-01-05,45.349998,NaN,NaN,NaN,45.284000,45.495998,44.389999,29558500
2017-01-06,45.801998,NaN,NaN,NaN,45.386002,46.062000,45.090000,27639500
2017-01-09,46.256001,NaN,NaN,NaN,45.793999,46.383999,45.599998,19897500
...,...,...,...,...,...,...,...,...
2022-05-02,902.940002,0.094719,0.016290,0.888990,860.770020,906.359985,848.030029,25260500
2022-05-03,909.250000,0.160966,0.011242,0.827792,903.179993,924.080017,888.590027,21236500
2022-05-04,952.619995,NaN,NaN,NaN,903.940002,955.500000,885.280029,27214600


In [24]:
stock_data.to_csv('/content/drive/MyDrive/JU PROJ/Reddit/EXEC_FINBERT_Reddit_data.csv')